6.5.2

In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Vila Velha,BR,2022-03-02 14:16:33,-20.3297,-40.2925,87.94,62,20,11.50
1,1,Xiaoyi,CN,2022-03-02 14:16:34,34.6178,109.7030,51.57,22,0,2.68
2,2,Broome,US,2022-03-02 14:16:34,42.2506,-75.8330,34.18,70,100,11.50
3,3,Rikitea,PF,2022-03-02 14:14:02,-23.1203,-134.9692,79.47,67,74,14.07
4,4,Bluff,NZ,2022-03-02 14:16:34,-46.6000,168.3333,43.81,84,61,6.35


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

6.5.3

In [6]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature (deg F) you would like for your trip? "))
max_temp = float(input("What is the maximum temperature (deg F) you would like for your trip? "))

In [7]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
8,8,Elizabeth City,US,2022-03-02 14:16:36,36.2946,-76.2511,58.05,69,0,8.05
14,14,Valencia,ES,2022-03-02 14:16:38,39.3333,-0.8333,71.89,33,78,18.97
16,16,Mulege,MX,2022-03-02 14:16:39,26.8833,-111.9833,61.95,28,93,2.95
20,20,Chhata,IN,2022-03-02 14:16:40,27.7167,77.5000,68.72,36,2,6.71
24,24,Hobart,AU,2022-03-02 14:13:09,-42.8794,147.3294,66.06,86,75,5.75
25,25,Makakilo City,US,2022-03-02 14:16:42,21.3469,-158.0858,69.28,84,0,4.61
29,29,Baton Rouge,US,2022-03-02 14:16:44,30.4508,-91.1545,56.93,70,96,1.01
32,32,Faya,SA,2022-03-02 14:16:45,18.3851,42.4509,71.71,45,13,9.93
33,33,Saldanha,ZA,2022-03-02 14:16:45,-33.0117,17.9442,71.02,75,17,12.30
34,34,Jalu,LY,2022-03-02 14:16:46,29.0331,21.5482,65.43,31,47,25.68


In [8]:
preferred_cities_df.count()

City_ID       78
City          78
Country       76
Date          78
Lat           78
Lng           78
Max Temp      78
Humidity      78
Cloudiness    78
Wind Speed    78
dtype: int64

In [9]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
8,Elizabeth City,US,58.05,36.2946,-76.2511,
14,Valencia,ES,71.89,39.3333,-0.8333,
16,Mulege,MX,61.95,26.8833,-111.9833,
20,Chhata,IN,68.72,27.7167,77.5000,
24,Hobart,AU,66.06,-42.8794,147.3294,
25,Makakilo City,US,69.28,21.3469,-158.0858,
29,Baton Rouge,US,56.93,30.4508,-91.1545,
32,Faya,SA,71.71,18.3851,42.4509,
33,Saldanha,ZA,71.02,-33.0117,17.9442,
34,Jalu,LY,65.43,29.0331,21.5482,


In [10]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [11]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
        
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

# Print 10 first hotels:
hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Lat,Lng,Hotel Name
8,Elizabeth City,US,58.05,36.2946,-76.2511,Hampton Inn Elizabeth City
14,Valencia,ES,71.89,39.3333,-0.8333,Masia El Pino
16,Mulege,MX,61.95,26.8833,-111.9833,Casa Granada
20,Chhata,IN,68.72,27.7167,77.5000,Raja earth movers
24,Hobart,AU,66.06,-42.8794,147.3294,St Ives Hobart Accommodation Tasmania
25,Makakilo City,US,69.28,21.3469,-158.0858,Marriott's Ko Olina Beach Club
29,Baton Rouge,US,56.93,30.4508,-91.1545,Hilton Baton Rouge Capitol Center
32,Faya,SA,71.71,18.3851,42.4509,املاك خالد الشعشوع
33,Saldanha,ZA,71.02,-33.0117,17.9442,Blue Bay Lodge
34,Jalu,LY,65.43,29.0331,21.5482,صاحبة الفخامة


In [12]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [14]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))